In [2]:
import os
import json
import pandas as pd
import traceback

In [13]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
key=os.getenv("GOOGLE_API_KEY")

In [16]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    api_key=key,
    temperature=0.2,
)